# Eat Safe, Love

## Notebook Set Up

In [22]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [23]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [24]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [25]:
# review the collections in our database
print(mongo.list_database_names())

['EPA', 'admin', 'autosauras', 'classDB', 'class_db', 'config', 'gardenDB', 'local', 'met', 'petsitly_marketing', 'test', 'travel_db', 'uk_food']


In [26]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [34]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene':20}
fields = {"BusinessName":1,"scores.Hygiene":1}
# Use count_documents to display the number of documents in the result
query_count = establishments.count_documents(query)
print("The number of documents in the result is ",query_count)
# Display the first document in the results using pprint
pprint(db.establishments.find_one(query, fields))

The number of documents in the result is  41
{'BusinessName': 'The Chase Rest Home',
 '_id': ObjectId('6497a39580b695e9e5e11692'),
 'scores': {'Hygiene': 20}}


In [28]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query, fields))
hygiene_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("The number of rows in the DataFrame is ", len(results))
# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

The number of rows in the DataFrame is  41


,_id,BusinessName,scores
0,6497a39580b695e9e5e11692,The Chase Rest Home,{'Hygiene': 20}
1,6497a39580b695e9e5e11a13,Brenalwood,{'Hygiene': 20}
2,6497a39580b695e9e5e11d1d,Melrose Hotel,{'Hygiene': 20}
3,6497a39580b695e9e5e11f0b,Seaford Pizza,{'Hygiene': 20}
4,6497a39580b695e9e5e11f1b,Golden Palace,{'Hygiene': 20}
5,6497a39580b695e9e5e128b6,Ashby's Butchers,{'Hygiene': 20}
6,6497a39580b695e9e5e12ada,South Sea Express Cuisine,{'Hygiene': 20}
7,6497a39580b695e9e5e14003,Golden Palace,{'Hygiene': 20}
8,6497a39580b695e9e5e14449,The Tulip Tree,{'Hygiene': 20}
9,6497a39580b695e9e5e14c59,F & S,{'Hygiene': 20}


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [39]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex': 'London'}, 'RatingValue': {'$gte': 4}}
fields = {"BusinessName":1,"RatingValue":1, 'LocalAuthorityName':1}
# Use count_documents to display the number of documents in the result
query_count = establishments.count_documents(query)
print("The number of documents in the result is ",query_count)

# Display the first document in the results using pprint
pprint(db.establishments.find_one(query, fields))

The number of documents in the result is  33
{'BusinessName': "Charlie's",
 'LocalAuthorityName': 'City of London Corporation',
 'RatingValue': 4,
 '_id': ObjectId('6497a39580b695e9e5e130a9')}


In [40]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query, fields))
rating_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("The number of rows in the DataFrame is ", len(results))
# Display the first 10 rows of the DataFrame
rating_df.head(10)

The number of rows in the DataFrame is  33


,_id,BusinessName,RatingValue,LocalAuthorityName
0,6497a39580b695e9e5e130a9,Charlie's,4,City of London Corporation
1,6497a39580b695e9e5e133d2,Mv City Cruises Erasmus,5,City of London Corporation
2,6497a39580b695e9e5e13f20,Benfleet Motor Yacht Club,4,City of London Corporation
3,6497a39580b695e9e5e14d20,Coombs Catering t/a The Lock and Key,5,City of London Corporation
4,6497a39580b695e9e5e14d22,Tilbury Seafarers Centre,5,City of London Corporation
5,6497a39580b695e9e5e15425,Mv Valulla,5,City of London Corporation
6,6497a39580b695e9e5e17534,Tereza Joanne,5,City of London Corporation
7,6497a39680b695e9e5e178fb,Brick Lane Brews,4,City of London Corporation
8,6497a39680b695e9e5e17918,City Bar & Grill,5,City of London Corporation
9,6497a39680b695e9e5e1791c,The Nuance Group (UK) Limited,5,City of London Corporation


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [74]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {'RatingValue': 5, "geocode.latitude":{'$lte': latitude + degree_search,'$gte': latitude - degree_search},"geocode.longitude":{'$lte': longitude + degree_search,'$gte': longitude - degree_search} }
fields = {"BusinessName":1,'LocalAuthorityName':1, "RatingValue":1,"scores.Hygiene":1 }
sort =  [('scores.Hygiene', 1)]
limit = 5
results = list(establishments.find(query, fields).sort(sort).limit(limit))
# Print the results
pprint(results)

[{'BusinessName': 'Volunteer',
  'LocalAuthorityName': 'Greenwich',
  'RatingValue': 5,
  '_id': ObjectId('6497a39580b695e9e5e16ef1'),
  'scores': {'Hygiene': 0}},
 {'BusinessName': 'Atlantic Fish Bar',
  'LocalAuthorityName': 'Greenwich',
  'RatingValue': 5,
  '_id': ObjectId('6497a39580b695e9e5e16f05'),
  'scores': {'Hygiene': 0}},
 {'BusinessName': 'Lumbini Grocery Ltd T/A Al-Iman',
  'LocalAuthorityName': 'Greenwich',
  'RatingValue': 5,
  '_id': ObjectId('6497a39580b695e9e5e16f08'),
  'scores': {'Hygiene': 0}},
 {'BusinessName': 'Iceland',
  'LocalAuthorityName': 'Greenwich',
  'RatingValue': 5,
  '_id': ObjectId('6497a39580b695e9e5e16ec4'),
  'scores': {'Hygiene': 0}},
 {'BusinessName': 'Howe and Co Fish and Chips - Van 17',
  'LocalAuthorityName': 'Greenwich',
  'RatingValue': 5,
  '_id': ObjectId('6497a39580b695e9e5e16ed3'),
  'scores': {'Hygiene': 0}}]


In [75]:
# Convert result to Pandas DataFrame
rating_df = pd.DataFrame(results)
# rating_df.sort_values(by='scores', ascending=True, inplace=True)
rating_df.head()

,_id,BusinessName,RatingValue,LocalAuthorityName,scores
0,6497a39580b695e9e5e16ef1,Volunteer,5,Greenwich,{'Hygiene': 0}
1,6497a39580b695e9e5e16f05,Atlantic Fish Bar,5,Greenwich,{'Hygiene': 0}
2,6497a39580b695e9e5e16f08,Lumbini Grocery Ltd T/A Al-Iman,5,Greenwich,{'Hygiene': 0}
3,6497a39580b695e9e5e16ec4,Iceland,5,Greenwich,{'Hygiene': 0}
4,6497a39580b695e9e5e16ed3,Howe and Co Fish and Chips - Van 17,5,Greenwich,{'Hygiene': 0}


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [81]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
match_query = {'$match': {'scores.Hygiene':0 }}
# 2. Groups the matches by Local Authority
group_query = {'$group': {'_id': "$LocalAuthorityName", 'count': {'$sum': 1}}}
# 3. Sorts the matches from highest to lowest
sort_values = {'$sort':{"count":-1, '_id':1}}

pipeline = [match_query, group_query, sort_values]
results = list(establishments.aggregate(pipeline))
# Print the number of documents in the result
print("Number of documents in result: ", len(results))

# Print the first 10 results
pprint(results[0:10])

Number of documents in result:  55
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [82]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(result_df))
# Display the first 10 rows of the DataFrame
result_df.head(10)

Rows in DataFrame:  55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
